In [12]:
import os
import json
import pandas as pd
from glob import glob

DATASET_NAME = "dataset_1"  # Nome do dataset, usado para construir o caminho

# Caminho da pasta contendo os arquivos .json
PRODUCTS_INFO_JSON_PATH = f"data/{DATASET_NAME}/products_info"  # ex: "./jsons"

# Lista para armazenar os dados extraídos
data = []

# Pega todos os arquivos .json da pasta
json_files = glob(os.path.join(PRODUCTS_INFO_JSON_PATH, "*.json"))

for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        try:
            file_data = json.load(f)
        except Exception as e:
            print(f"Erro ao carregar {file_path}: {e}")
            continue

        for entry in file_data:
            results = entry.get("results", [])
            for product in results:
                product_id = product.get("id")
                name = product.get("name", "")
                attributes_raw = product.get("attributes", [])

                # Converte atributos para dicionário simples: {name: value_name}
                attributes = {
                    attr.get("name"): attr.get("value_name")
                    for attr in attributes_raw
                    if attr.get("name") and attr.get("value_name")
                }

                data.append({
                    "PRODUCT_ID": product_id,
                    "TITLE": name,
                    "ATTRIBUTES": attributes,
                    "SOURCE_FILE": os.path.basename(file_path)
                })

# Criar o DataFrame final
df = pd.DataFrame(data)

# Visualizar as primeiras linhas
print(df.head())

# Salvar em CSV ou Parquet se quiser
df.to_csv(f"data/{DATASET_NAME}/{DATASET_NAME}.csv", index=False)


    PRODUCT_ID                                              TITLE  \
0  MLB29523289  Base Estrela Secretaria Em Aço C/ Rodízios E C...   
1  MLB34230359  Escrivaninha Office Nt2105 Notável Móveis Cor ...   
2  MLB15904595    Cadeira de Escritório Mymax Gamer MX5 Cor Preto   
3  MLB26801865  Marmita Dupla Camada Á Prova De Vazamento Imed...   
4  MLB22522670  Cadeira de escritório TGTTGT-OFC-001 TGT ergon...   

                                          ATTRIBUTES  \
0  {'Marca': 'cadeira giratória diretora', 'Fabri...   
1  {'Marca': 'Notável Móveis', 'Modelo': 'NT2105'...   
2  {'Marca': 'Mymax', 'Linha': 'Gamer', 'Modelo':...   
3  {'Marca': 'Marmita,Lancheira,Escritorio', 'Mod...   
4  {'Marca': 'TGT', 'Linha': 'cadeira fixa escrit...   

                        SOURCE_FILE  
0  info_products_10_escritório.json  
1  info_products_10_escritório.json  
2  info_products_10_escritório.json  
3  info_products_10_escritório.json  
4  info_products_10_escritório.json  


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12648 entries, 0 to 12647
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PRODUCT_ID   12648 non-null  object
 1   TITLE        12648 non-null  object
 2   ATTRIBUTES   12648 non-null  object
 3   SOURCE_FILE  12648 non-null  object
dtypes: object(4)
memory usage: 395.4+ KB


In [14]:
df.groupby('source_file').count()

KeyError: 'source_file'